# テーブル概要
accident:  
event:  
gv(general vehicle):車両一般  
ve(Exterior Vehicle):車両外部  
vi(Interior Vehicle):車両内部  
oa(OCCUPANT ASSESSMENT):乗員の調査  
oi(OCCUPANT INJURY):乗員の傷害(mergeに使用できるkeyの値が同一でも傷害箇所によってレコードが増加)  

# 最終的な作成データ
- Crash year 2010–2015
- Vehicle model year 2001–2015
- Light vehicles (passenger cars, pick-ups and mini-vans) 
- Non-ejected occupants
- Occupant age 15 or higher
- Occupants with known injury status or fatality

# ライブラリのインポート Pandasの表示設定
同一cellに複数テーブルを表示  
全カラムを表示  
最大表示行数:500  
1つのカラムの最大表示文字数:200  
floatの有効桁数:4  
色付き文字の出力:print(pycolor.RED + '文字列' + pycolor.END)  

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import csv
import sys
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (roc_curve, auc, accuracy_score)
from sklearn.linear_model import Lasso
from IPython import embed
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 200)
pd.options.display.float_format = '{:.4g}'.format
class pycolor:
    BLACK = '\033[30m'
    RED = '\033[31m'
    GREEN = '\033[32m'
    YELLOW = '\033[33m'
    BLUE = '\033[34m'
    PURPLE = '\033[35m'
    CYAN = '\033[36m'
    WHITE = '\033[37m'
    END = '\033[0m'
    BOLD = '\038[1m'
    UNDERLINE = '\033[4m'
    INVISIBLE = '\033[08m'
    REVERCE = '\033[07m'

# NASS CDSデータの読み込み

In [ ]:
path = os.path.dirname(os.path.abspath('__file__'))
file_name = ['accident', 'event', 'gv', 'oa', 'oi', 've', 'vi']
cds_key = []
cds = {}
uyear = [str(x) for x in range(1, 16)]
for year in range(2001, 2016):
    for file in file_name:
        if year >= 2009:
            df = pd.read_sas(os.path.join(path, str(year), 'FormattedData', '{}.sas7bdat'.format(file)))
        elif year >= 2001:
            df = pd.read_sas(os.path.join(path, str(year), 'PCSAS', '{}.sas7bdat'.format(file)))
        cds_key.append('{}_{}'.format(file, year - 2000))
        cds['{}_{}'.format(file, year - 2000)] = df

In [ ]:
path = os.path.dirname(os.path.abspath('__file__'))
expand_name = ['airbag', 'bagseat', 'childseat', 'seatloc', 'tire', 'tiredmg']
exp_key = []
exp = {}
for year in range(2011, 2016):
    for expand in expand_name:
        try:
            df = pd.read_sas(os.path.join(path, str(year), 'ExpandedSAS', '{}.sas7bdat'.format(expand)))
        except FileNotFoundError:
            df = pd.read_sas(os.path.join(path, str(year),  'ExpandedSAS', 'UNFORMATTED', '{}.sas7bdat'.format(expand)))
        exp_key.append('{}_{}'.format(expand, year - 2000))
        exp['{}_{}'.format(expand, year - 2000)] = df

# 特定のカラムが存在するか確認
引数：DFを格納した辞書, 有無を確かめたいカラムのリスト, オプション:調べたいDFのkey(デフォルトは全てのDF)

In [ ]:
def exist_check(df_dic, check_columns, df_keys = None):
    if df_keys is None:
        df_keys = df_dic.keys()
    return_df = pd.DataFrame()
    for key in df_keys:
        df_columns = df_dic[key].columns.values
        check_dic = {}
        for check_column in check_columns:
            check_flag = 0
            for df_column in df_columns:
                if df_column == check_column:
                    check_flag = 1
            check_dic[check_column] = check_flag
        check_df = pd.io.json.json_normalize(check_dic)
        check_df = check_df.rename(index = {0: key})
        return_df = pd.concat([return_df, check_df])
    return_df.head(len(return_df))
    return return_df

# 同名のDFで，年毎に異なるカラムを持つか確認
年によって異なるカラムを持つファイル名とそのカラム名を出力する

In [ ]:
def year_check(df_dic):
    df_keys = df_dic.keys()
    return_dic = {}
    name_list = []
    for key in df_keys:
        split_key = key.split('_')
        name = split_key[0]
        year = split_key[1]
        df_columns = df_dic[key].columns.values.tolist()
        if name not in name_list:
            name_list.append(name)
            return_dic[name] = pd.DataFrame()
        for column in df_columns:
            return_dic[name].loc[year, column] = 1
    for name in name_list:
        if return_dic[name].isnull().values.sum() != 0:
            print(pycolor.RED + name + pycolor.END)
            column_list = return_dic[name].columns.values.tolist()
            column_series = return_dic[name].isnull().any()
            for column in column_list:
                if column_series[column] == True:
                    print(column)
    return return_dic

#  テーブル間で重複するカラムの出力
デフォルトで重複は15年について調べる

In [ ]:
def same_column(df_dic, conf_year = '15'):
    df_keys = df_dic.keys() 
    column_dic = {}
    name_list = []
    duplicate_list = []
    return_df = pd.DataFrame()
    for key in df_keys:
        name = key.split('_')[0]
        year = key.split('_')[1]
        if year == conf_year:
            column_dic[name] = df_dic[key].columns.values.tolist()
            name_list.append(name)
    for name in name_list:
        duplicate_list.extend(column_dic[name])
    duplicate_list = [x for x in set(duplicate_list) if duplicate_list.count(x) > 1]
    for name in name_list:
        for column in column_dic[name]:
            if column in duplicate_list:
                return_df.loc[name, column] = 1
    return return_df

# 重複したレコードの抽出
引数：DF辞書, DFのkey, 重複判定column(リスト)   
重複判定columnを先頭にしたDFを出力  

In [ ]:
def duplicate(df_dic, dic_key, df_key):
    df = df_dic[dic_key]
    df = df[df.duplicated(subset = df_key, keep = False)]
    columns = list(df.columns.values)
    for column in df_key:
        columns.remove(column)
    return df.loc[:, df_key + columns]

In [ ]:
cds_column = year_check(cds)

In [ ]:
cds_column['oa'][['CHOWUSED', 'LATCHDES', 'LATCHUSE', 'POSPRES', 'POSUSE', 'POSGUIDE']]
cds_column['ve'][['TOWRES']]

In [ ]:
cds['accident_11'].head()

In [ ]:
cds['event_11'].head()

In [ ]:
cds['gv_15'].head()

In [ ]:
cds['gv_15'].describe()
len(cds['gv_15'])

In [ ]:
cds['oi_11'].head()

In [ ]:
cds['oa_15'].head()

In [ ]:
len(pd.merge(cds['oa_10'], cds['gv_10'], on = ['CASEID', 'CASENO', 'PSU', 'RATWGT', 'STRATIF', 'VERSION', 'VEHNO'], how = 'inner'))
print(len(cds['gv_10']), len(cds['oa_10']))

In [ ]:
cds['ve_11'].head()

In [ ]:
cds['vi_11'].head()

## 整数の値が入っているはずなのに小数点以下の値が入っているもの，同じ値なのにpython内部で別の値として認識されているものを修正
## 複数のテーブルで重複しているが利用しないカラムの削除

In [ ]:
for col in ['PSU', 'VEHNO']:
    for y in uyear:
        for og in ['oa', 'gv', 've', 'vi']:
            cds['{}_{}'.format(og, y)][col] = cds['{}_{}'.format(og, y)][col].astype(np.int64)
        if col !=  'VEHNO':
            og = 'accident'
            cds['{}_{}'.format(og, y)][col] = cds['{}_{}'.format(og, y)][col].astype(np.int64)
            
for y in uyear:
    y  = str(y)
    cds['accident_{}'.format(y)]['VEHFORMS'] = cds['accident_{}'.format(y)]['VEHFORMS'].astype(np.int64)

for col in ['CASENO', 'RATWGT', 'STRATIF', 'VERSION']:
    for y in range(10, 16):
        for og in ['oa', 'gv', 've', 'vi', 'accident', 'event', 'oi']:
            cds['{}_{}'.format(og, y)] = cds['{}_{}'.format(og, y)].drop(col, axis = 1)

## テーブル，年毎のレコード数を表示

In [ ]:
for year in uyear:
    print(year)
    for file in file_name:
        print('{}:'.format(file) + str(len(cds['{}_{}'.format(file, year)])))

## 年毎に同一のカラムを表示

In [ ]:
for year in uyear:
    print(year)
    same_column(cds, str(year)).head(7)

## テーブル毎の結合keyを宣言

In [ ]:
merge_key = {}
for file in file_name:
    if file == 'accident' or file == 'event':
        merge_key[file] =  ['CASEID', 'PSU']
    if file == 'gv' or file == 've' or file == 'vi':
        merge_key[file] = ['CASEID', 'PSU', 'VEHNO']
    if file == 'oa' or file == 'oi':
        merge_key[file] = ['CASEID', 'PSU', 'VEHNO', 'OCCNO']

## 同一の結合keyに対して重複しているレコードを表示

In [ ]:
dup_dic = {}
for file in file_name:
    print(file)
    dup_dic[file] = duplicate(cds, '{}_10'.format(file), merge_key[file])
    dup_dic[file].head()

## テーブルの結合
oi,eventテーブルを除き,2010~2015年のデータをそれぞれ結合する

In [ ]:
cds_merge = {}
for year in uyear:
    cds_merge[year] = cds['oa_{}'.format(year)]
    for file in [x for x in file_name if not (x == 'oa' or x == 'oi' or x == 'event')]:
        if file != 'gv':
            cds_merge[year] = pd.merge(cds_merge[year], cds['{}_{}'.format(file, year)], on = merge_key[file], how = 'left')
        else:
            cds_merge[year] = pd.merge(cds_merge[year], cds['{}_{}'.format(file, year)], on = merge_key[file], how = 'inner')
    print(year)
    print('oa_length:' + str(len(cds['oa_{}'.format(year)])))
    print('merge_length:' + str(len(cds_merge[year])))
    cds_merge[year].head(5)

## 結合後の編集用辞書データの作成

In [ ]:
cds_prepro = {}
for year in uyear:
    cds_prepro[year] = cds_merge[year]
    print(year,len(cds_prepro[year]))

## 利用するカラムを抽出
pcol:目的変数  
CASEID, PSU, VEHNO: 結合キー  
VEHFORMS: 事故に関わった車両数  
BODYTYPE, CURBWGT, TRAVELSP: 自車データ  
PDOF1, GAD1, SHL1: 衝突箇所データ(角度, 位置(前後左右上下), 位置(GAD1と共に使うことで前後左右の面をそれぞれ3分割できる))  
otbdytyp, otvehwgt: 相手車両データ  
BAGAVAIL, PARUSE: エアバッグの有無・シートベル着用の有無  
AGE, SEX, HEIGHT, WEIGHT: 乗員データ:一般に重症度に大きな影響があると言われる4つの要素  

In [ ]:
pcol = 'MAIS'
use_col = [pcol, \
           'CASEID', 'PSU', 'VEHNO', \
           'VEHFORMS', \
           'BODYTYPE', 'CURBWGT', 'TRAVELSP', \
           'PDOF1', 'GAD1', \
           'otbdytyp', 'otvehwgt', \
           'BAGAVAIL', 'PARUSE', \
           'AGE', 'SEX', 'HEIGHT', 'WEIGHT']
for year in uyear:
    cds_prepro[year] = cds_prepro[year][use_col]

## もう一方の車両の速度を示すカラムを作成

In [ ]:
for year in uyear:
    sp = cds_prepro[year][['CASEID', 'PSU', 'VEHNO', 'TRAVELSP']]
    sp = sp.drop_duplicates(subset = ['CASEID', 'PSU', 'VEHNO'])
    sp1 = sp.query('VEHNO == 1')
    sp1 = sp1.drop('VEHNO', axis  = 1)
    sp1 = sp1.rename(columns = {'TRAVELSP': 'SP1'})
    sp2 = sp.query('VEHNO == 2')
    sp2 = sp2.drop('VEHNO', axis  = 1)
    sp2 = sp2.rename(columns = {'TRAVELSP': 'SP2'})
    sp = pd.merge(sp1, sp2, on = ['CASEID', 'PSU'], how = 'inner')
    v1o = sp[['CASEID', 'PSU', 'SP2']]
    v1o['VEHNO'] = 1
    v2o = sp[['CASEID', 'PSU', 'SP1']]
    v2o['VEHNO'] = 2   
    cds_prepro[year] = pd.merge(cds_prepro[year], v1o, on = ['CASEID', 'PSU', 'VEHNO'], how = 'left')
    cds_prepro[year] = pd.merge(cds_prepro[year], v2o, on = ['CASEID', 'PSU', 'VEHNO'], how = 'left')
    cds_prepro[year] = cds_prepro[year].fillna({'SP1': 0,  'SP2':  0})
    cds_prepro[year]['otbsp'] = cds_prepro[year]['SP1'] + cds_prepro[year]['SP2'] 
    cds_prepro[year] = cds_prepro[year].drop(['SP1', 'SP2'], axis = 1)

## NANを含むレコードを削除

In [ ]:
for year in uyear:
    print(year)
    print('削除前：', len(cds_prepro[year]))
    cds_prepro[year] = cds_prepro[year].dropna(how = 'any')
    print('削除後：', len(cds_prepro[year]))

## CASEID以外をintに

In [ ]:
use_col = [x for x in use_col if x != 'CASEID' and x != 'GAD1']
for col in use_col:
    for year in uyear:
        cds_prepro[year][col] = cds_prepro[year][col].astype(np.int64)

## Car to Carの事故に限定
2台の事故のみに変更

In [ ]:
for year in uyear:
    cds_prepro[year] = cds_prepro[year].query('VEHFORMS == 2')
    print(year, len(cds_prepro[year]))

## 値は入っているがUnknownを示す値が入っているものを削除

In [ ]:
for year in uyear:
    print(pycolor.RED + year + pycolor.END)
    print(len(cds_prepro[year]))
    cds_prepro[year] = cds_prepro[year].query('TRAVELSP < 777 and CURBWGT < 998 and GAD1 != 9 and PDOF1 < 998 and otvehwgt < 998 and PARUSE != 10 and otbsp < 777')
    print(len(cds_prepro[year]))

## シートベルト使用者とエアバッグ利用可能だった乗員に限定

In [ ]:
for year in uyear:
    print(pycolor.RED + year + pycolor.END)
    print(len(cds_prepro[year]))
    cds_prepro[year] = cds_prepro[year].query('BAGAVAIL == 1 and PARUSE != 0')
    print(len(cds_prepro[year]))

## 車両毎にMAISの最大値のみ残す
MAISでソートし, VEHNOが重複しているレコードを上にあるものを残して削除

In [ ]:
for year in uyear:
    print(pycolor.RED + year + pycolor.END)
    print(len(cds_prepro[year]))
    cds_prepro[year] = cds_prepro[year].sort_values(by = pcol, ascending = False)
    cds_prepro[year] = cds_prepro[year].drop_duplicates(subset = ['CASEID', 'PSU', 'VEHNO'])
    print(len(cds_prepro[year]))

## 結合や絞り込みに必要だったカラムの除外

In [ ]:
for year in uyear:
    cds_prepro[year] = cds_prepro[year].drop(['CASEID', 'PSU', 'VEHNO', 'BAGAVAIL', 'PARUSE', 'VEHFORMS'], axis = 1)

## 性別を男女にする

In [ ]:
for year in uyear:
    cds_prepro[year].loc[cds_prepro[year]['SEX'] >= 2, 'SEX']  = 0

## ダミー変数に変換

In [ ]:
dummy = pd.concat([cds_prepro['1'], cds_prepro['2'], cds_prepro['3'], cds_prepro['4'], cds_prepro['5'], cds_prepro['6'], cds_prepro['7'], cds_prepro['8'], cds_prepro['9'], cds_prepro['10'], cds_prepro['11'], cds_prepro['12'], cds_prepro['13'], cds_prepro['14'], cds_prepro['15']])
dummy = pd.get_dummies(dummy, drop_first = True, columns = ['BODYTYPE', 'GAD1', 'otbdytyp', 'SEX'])

In [ ]:
dummy.head()

## 最終的なデータ数と元のデータ数

In [ ]:
length = 0
for year in uyear:
    length += len(cds['gv_{}'.format(year)])
print('元データ:', length)
print('最終：',len(dummy))

## テストデータと訓練データに分割
時系列を考慮して15年のデータをテストデータとする

In [ ]:
dummy_15 = dummy[len(dummy) - len(cds_prepro['15']):]
dummy_b14 = dummy[:len(dummy) - len(cds_prepro['15'])]
X_train = dummy_b14.drop([pcol], axis = 1)
y_train = dummy_b14[pcol]
X_test = dummy_15.drop([pcol], axis = 1)
y_test = dummy_15[pcol]

## ランダムフォレストで分類

In [ ]:
forest = RandomForestClassifier(n_estimators = 1000, random_state = 0)
forest.fit(X_train, y_train)
predict = forest.predict(X_test)
fpr, tpr, thresholds = roc_curve(y_test, predict, pos_label = 1)
auc(fpr, tpr)
accuracy_score(predict, y_test)
fti = forest.feature_importances_
columns = dummy.columns.values
columns = list(columns)
columns.remove('MAIS')
print('Feature Importance')
for i, col in enumerate(columns):
    print(col, ':', fti[i])

## 15年の要約統計量

In [ ]:
cds_prepro['15'].describe()

In [ ]:
cds_prepro['15'].head(10)

# exp

In [ ]:
#exp_column = year_check(exp)

In [ ]:
'''
pd.DataFrame(exp_column['airbag']['RECTYPE'])
pd.DataFrame(exp_column['bagseat']['RECTYPE'])
exp_column['childseat'][['STRATIF','VERSION']]
exp_column['seatloc'][['VERSION' ,'RECTYPE' ,'POSGUIDE']]
'''

In [ ]:
#exp['airbag_11']

In [ ]:
'''
for expand in expand_name:
    print('{}:'.format(expand) + str(len(exp['{}_15'.format(expand)])))
'''

In [ ]:
#same_column(exp)

In [ ]:
'''
for expand in expand_name:
    print('{}:'.format(expand) + str(len(exp['{}_15'.format(expand)])))
'''

In [ ]:
#same_column(exp)

#  以下挙動確認用

In [ ]:
hoge = pd.DataFrame({'key1':[1, 1, 1],
                     'key2':[3,3,3],
                     'key3':[1, 3, 3],
                     'data_x':['a', 'b', 'c']})
hoge.head()
hoge2 = pd.DataFrame({'key1':[1, 1, 3],
                      'key2':[3, 3, 5],
                      'key3':[1, 3, 4],
                      'data_y':['d', 'e', 'f']})
hoge2.head()
hoge3 = pd.DataFrame({'key1':[1],
                      'data':['g']})
hoge3.head()

In [ ]:
pd.merge(hoge, hoge3, on = 'key1').head()

In [ ]:
hoge3 = pd.merge(hoge, hoge2, on = 'key1', how = 'outer')

In [ ]:
hoge3.head(10)

In [ ]:
hoge3['key2_x'] = hoge3['key2_x'] + 1

In [ ]:
hoge3.head(10)

In [ ]:
hoge['data_x'] = hoge['data_x'].replace('a', 'e')

In [ ]:
hoge.head()

In [ ]:
#pd.merge(hoge, hoge2, on = ['key1', 'key2'], how = 'outer')
#pd.merge(hoge, hoge2, on = ['key1', 'key2'], how = 'inner')
pd.merge(hoge, hoge2, on = ['key1', 'key2', 'key3'], how = 'left')

In [ ]:
test = pd.merge(hoge, hoge2, on = 'key1', how = 'inner')
test.head()
test = pd.merge(test, hoge3, on = 'key1', how = 'inner')
test.head(10)

In [ ]:
hogehoge = pd.DataFrame()
pd.merge(hoge, hogehoge, on = ['key1'])

## 利用するか不明

## 特徴量のドロップ
結合用の特徴量・AISを元にした特徴量・年によってデータが存在しない特徴量をドロップ

In [ ]:
'''
cds_prepro = {}
drop_columns = ['CASEID', 'CASENO', 'VEHNO', 'OCCNO', 'RATWGT', 'STRATIF', 'VERSION', 'AAIS',\
                'AAIS08', 'VAIS', 'VAIS08', 'MAIS', 'MAIS08', 'INJNUM08', 'ISS08', 'TOWRES', \
                'VIN', 'AINJSER', 'AINJSER8', 'AINJURED', 'AINJURD8', 'VINJSER', 'VINJSER8', \
                'VINJURED', 'VINJURD8', 'INJNUM', 'LATCHDES', 'LATCHUSE', 'POSPRES', 'POSUSE', \
                'POSGUIDE','CHOWUSED', 'FUELTYP1', 'FUELTYP2']#INJSEVは警察の報告でありAISデータを使用していない
for year in range(10, 16):
    year = str(year)
    cds_prepro[year] = cds_merge[year]
    year_columns = cds_prepro[year].columns.values.tolist()
    for column in drop_columns:
        if column in year_columns:
            cds_prepro[year] = cds_prepro[year].drop(columns = [column])
'''

## ISSデータが存在しないレコードのドロップ
推定対象のISSが存在しないレコードのドロップ

In [ ]:
#ISSのnanをdropし，年毎のレコード数をprint
'''
for year in range(10, 16):
    year = str(year)
    cds_prepro[year] = cds_prepro[year].dropna(subset = ['ISS'])
    print(str(year) + ':' + str(len(cds_prepro[year])))
'''

## 論文と年以外の条件を同じに
Vehicle Modelの0~490が相当

In [ ]:
'''
q_column = ['AGE', 'EJECTION', 'MODEL', 'MODELYR']
query = ['AGE >= 15', 'EJECTION == 0', '0 <= MODEL <= 490', '2001 <= MODELYR <= 2015']
for year in range(10, 16):
    year =  str(year)
    print(pycolor.GREEN + 'year:' + year + pycolor.END)
    print('レコード数:', len(cds_prepro[year]))
    for col, que in zip(q_column, query):
        print(pycolor.RED +'column:' + col + pycolor.END)
        print('NaN:', cds_prepro[year][col].isnull().sum())
        print('クエリによるレコードの減少数(' + que + '):', len(cds_prepro[year]) - len(cds_prepro[year].query(que)))
    print()
'''

In [ ]:
'''
query = ['AGE >= 15', 'EJECTION == 0']#, '0 <= MODEL <= 490', '2001 <= MODELYR <= 2015']
for que in query:
    print('query:', que)
    print('クエリによる減少数(残ったレコード数)')
    for year in range(10, 16):
        year = str(year)
        before = len(cds_prepro[year])
        cds_prepro[year] = cds_prepro[year].query(que)
        print(' ', year, ':', str(before - len(cds_prepro[year])), '({})'.format(len(cds_prepro[year])))
        #print(' ', year, ':', str(before - len(cds_prepro[year].query(que))))
'''

In [ ]:
#cds_prepro['15']['VEHWGT'].isnull().sum()

## カラム名をCSVに出力

In [ ]:
'''
col = cds_prepro['15'].query('MODEL > 500')
path = os.path.dirname(os.path.abspath('__file__'))
col.to_csv(os.path.join(path, 'column.csv'))
'''

## 値の変更
同一のものを指しているにも関わらず年によって値が違うものを修正

In [ ]:
"""
for year in range(10, 16):
    year = str(year)
    cds_prepro[year]['CLIMATE'] = cds_prepro[year]['CLIMATE'].replace('13', '20') 
"""